In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('Amazon - Movies and TV Ratings.csv')

In [3]:
df.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AH3QC2PC1VTGP,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3LKP6WPMP9UKX,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVIY68KEPQ5ZD,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1CV1WROP5KTTW,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#maximum views/ratings
df.describe().T['count'].sort_values(ascending=False)[:1].to_frame()

,count
Movie127,2313.0


Movie127 have maximum ratings

In [7]:
# Average rating of each movie
df.drop('user_id',axis=1).mean()

Movie1      5.000000
Movie2      5.000000
Movie3      2.000000
Movie4      5.000000
Movie5      4.103448
              ...   
Movie202    4.333333
Movie203    3.000000
Movie204    4.375000
Movie205    4.628571
Movie206    4.923077
Length: 206, dtype: float64

In [9]:
# Top 5 Movies with maximum ratings
df.drop('user_id',axis=1).mean().sort_values(ascending=False)[:5].to_frame()

,0
Movie1,5.0
Movie66,5.0
Movie76,5.0
Movie75,5.0
Movie74,5.0


In [10]:
# Top 5 movies with least audience
df.describe().T['count'].sort_values(ascending=True)[:5].to_frame()

,count
Movie1,1.0
Movie71,1.0
Movie145,1.0
Movie69,1.0
Movie68,1.0


Model building

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')


In [14]:
data_melt = df.melt(id_vars = df.columns[0],value_vars=df.columns[1:],
                            var_name="Movies",value_name="Rating")
data_melt

,user_id,Movies,Rating
0,A3R5OBKS7OM2IR,Movie1,5.0
1,AH3QC2PC1VTGP,Movie1,NaN
2,A3LKP6WPMP9UKX,Movie1,NaN
3,AVIY68KEPQ5ZD,Movie1,NaN
4,A1CV1WROP5KTTW,Movie1,NaN
...,...,...,...
998683,A1IMQ9WMFYKWH5,Movie206,5.0
998684,A1KLIKPUF5E88I,Movie206,5.0
998685,A5HG6WFZLO10D,Movie206,5.0
998686,A3UU690TWXCG1X,Movie206,5.0


In [18]:
dataset=data_melt.fillna(0)

In [19]:
dataset

,user_id,Movies,Rating
0,A3R5OBKS7OM2IR,Movie1,5.0
1,AH3QC2PC1VTGP,Movie1,0.0
2,A3LKP6WPMP9UKX,Movie1,0.0
3,AVIY68KEPQ5ZD,Movie1,0.0
4,A1CV1WROP5KTTW,Movie1,0.0
...,...,...,...
998683,A1IMQ9WMFYKWH5,Movie206,5.0
998684,A1KLIKPUF5E88I,Movie206,5.0
998685,A5HG6WFZLO10D,Movie206,5.0
998686,A3UU690TWXCG1X,Movie206,5.0


In [20]:
x_train,x_test=train_test_split(dataset,test_size=0.3,random_state=0)


In [22]:
x_train.shape

(699081, 3)

In [24]:
x_test.shape

(299607, 3)

In [25]:
cf=pd.concat([x_train,x_test]).reset_index()
cf.head()

,index,user_id,Movies,Rating
0,338560,A9Q5O3PAC51MV,Movie70,0.0
1,471992,A1EI65WJC85U68,Movie98,0.0
2,185811,A379SAP75SPDHD,Movie39,0.0
3,817860,A1JIPFV4OL520T,Movie169,0.0
4,251599,A11YEGV0NPRF3H,Movie52,0.0


In [29]:
pvt=cf.pivot(index='user_id',columns='Movies',values='Rating')
pvt.head()

Movies,Movie1,Movie10,Movie100,Movie101,Movie102,Movie103,Movie104,Movie105,Movie106,Movie107,...,Movie90,Movie91,Movie92,Movie93,Movie94,Movie95,Movie96,Movie97,Movie98,Movie99
user_id,,,,,,,,,,,,,,,,,,,,,
A0047322388NOTO4N8SKD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00473363TJ8YSZ3YAGG9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1004AX2J2HXGL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100CQXJ6D44T9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100Z2S0880G9A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
pvt.shape

(4848, 206)

In [32]:
pvt['user_index']=np.arange(0,pvt.shape[0],1)
pvt.head()

Movies,Movie1,Movie10,Movie100,Movie101,Movie102,Movie103,Movie104,Movie105,Movie106,Movie107,...,Movie91,Movie92,Movie93,Movie94,Movie95,Movie96,Movie97,Movie98,Movie99,user_index
user_id,,,,,,,,,,,,,,,,,,,,,
A0047322388NOTO4N8SKD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
A00473363TJ8YSZ3YAGG9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
A1004AX2J2HXGL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
A100CQXJ6D44T9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
A100Z2S0880G9A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [33]:
pvt.set_index(['user_index'],inplace=True)
pvt

Movies,Movie1,Movie10,Movie100,Movie101,Movie102,Movie103,Movie104,Movie105,Movie106,Movie107,...,Movie90,Movie91,Movie92,Movie93,Movie94,Movie95,Movie96,Movie97,Movie98,Movie99
user_index,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# Singular Value Secomposition

In [35]:
u, sigma, vt = svds(pvt,k=10)

In [37]:
#left singular matrix
u

array([[-1.41948014e-07,  0.00000000e+00,  3.70967613e-07, ...,
         1.50829005e-07, -4.26840615e-02, -7.96633419e-05],
       [ 1.35316796e-05,  5.13279606e-18, -3.46952163e-05, ...,
        -8.11610274e-06,  1.14463559e-04, -1.91174114e-02],
       [ 3.38291990e-06,  1.28319901e-18, -8.67380407e-06, ...,
        -2.02902569e-06,  2.86158897e-05, -4.77935284e-03],
       ...,
       [ 5.83560217e-17, -5.12079788e-17, -2.15057739e-18, ...,
         2.80729933e-18,  1.12638706e-18, -9.77882255e-19],
       [ 1.01487597e-05,  3.84959704e-18, -2.60214122e-05, ...,
        -6.08707706e-06,  8.58476690e-05, -1.43380585e-02],
       [ 6.76583981e-06,  2.56639803e-18, -1.73476081e-05, ...,
        -4.05805137e-06,  5.72317793e-05, -9.55870568e-03]])

In [38]:
sigma

array([ 34.72750225,  39.54231658,  41.16418185,  47.74283509,
        53.94837751,  75.614235  ,  77.65328486,  82.08093229,
       117.1348772 , 209.22865684])

In [39]:
# diagonal matrix
sigma=np.diag(sigma)
sigma

array([[ 34.72750225,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,  39.54231658,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,  41.16418185,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,  47.74283509,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         53.94837751,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  75.614235  ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  

In [40]:
#Right singular matrix
vt

array([[ 8.80742461e-18,  4.12188543e-20, -5.14725821e-17, ...,
         4.29876196e-17,  1.17053737e-17, -2.86577494e-17],
       [-6.04778802e-18, -2.69716801e-20,  3.28390103e-17, ...,
        -4.90601570e-17, -9.26786201e-18,  2.06991882e-17],
       [-1.54140315e-17,  3.01696910e-20, -5.19119135e-17, ...,
         1.04465392e-16,  8.13516843e-18, -6.54065377e-17],
       ...,
       [ 2.47848977e-18,  7.59626632e-21, -7.28740593e-18, ...,
        -4.24420858e-18,  1.80783071e-18, -5.70492039e-20],
       [ 1.80479109e-17,  4.48892113e-20, -5.02959782e-17, ...,
        -3.18575876e-17,  1.35694779e-17,  1.46018920e-17],
       [ 5.50504088e-17,  1.18723560e-19, -1.28576337e-16, ...,
        -1.06599787e-16,  3.57470449e-17,  4.97935872e-17]])

In [41]:
# Prediction
predicted_rating=np.dot(np.dot(u,sigma),vt)
pred=pd.DataFrame(predicted_rating,columns=pvt.columns)
pred.head()

Movies,Movie1,Movie10,Movie100,Movie101,Movie102,Movie103,Movie104,Movie105,Movie106,Movie107,...,Movie90,Movie91,Movie92,Movie93,Movie94,Movie95,Movie96,Movie97,Movie98,Movie99
0,-9.128838e-17,-2.272035e-19,2.545680e-16,1.817148e-16,6.498682e-07,1.093853e-06,4.345189e-12,-2.497853e-09,-2.506972e-09,-0.000003,...,5.399395e-07,2.204105e-07,1.330251e-06,1.288095e-16,3.514101e-17,0.000001,1.482448e-16,1.606660e-16,-6.867427e-17,-7.357047e-17
1,-2.200700e-16,-4.753755e-19,5.150589e-16,5.005842e-16,-6.889186e-05,-2.419713e-06,-9.709430e-10,-3.222229e-07,-3.202302e-07,0.000316,...,-5.695961e-05,-2.481068e-06,7.615735e-06,2.828954e-16,8.567926e-17,-0.000137,3.616540e-16,4.250451e-16,-1.431230e-16,-1.983919e-16
2,-5.501750e-17,-1.188439e-19,1.287647e-16,1.251460e-16,-1.722296e-05,-6.049284e-07,-2.427357e-10,-8.055573e-08,-8.005755e-08,0.000079,...,-1.423990e-05,-6.202669e-07,1.903934e-06,7.072385e-17,2.141981e-17,-0.000034,9.041350e-17,1.062613e-16,-3.578075e-17,-4.959797e-17
3,4.274205e-17,2.554430e-19,-3.106092e-16,1.845739e-16,1.932794e-06,7.188264e-08,-1.267691e-10,-3.464287e-08,-3.439380e-08,-0.000010,...,1.617642e-06,7.717334e-08,-2.259384e-07,-7.598565e-17,1.776231e-18,0.000004,-1.359844e-17,1.322711e-16,7.579251e-17,-8.904957e-17
4,1.147485e-17,3.105607e-20,-2.720744e-17,-1.874918e-17,6.549731e-05,-3.995040e-05,-4.219027e-08,-4.507963e-06,-4.425759e-06,-0.000409,...,5.393707e-05,-1.041088e-04,-1.538701e-04,-1.221829e-17,-3.362624e-18,0.000089,-1.927397e-17,-2.608671e-17,7.004777e-18,2.888517e-18


In [52]:
def rec_items(user_index,pvt,pred,num_rec):
    user_idx=user_index
    sorted_user_rating=pvt.iloc[user_idx].sort_values(ascending=False)
    sorted_user_prediction=pred.iloc[user_idx].sort_values(ascending=False)
    
    temp=pd.concat([sorted_user_rating,sorted_user_prediction],axis=1)
    temp.index.name='Recommended Movie'
    temp.columns=['user_ratings','user_predictions']
    temp=temp.loc[temp.user_ratings==0]
    temp=temp.sort_values('user_predictions',ascending=False)
    print('\nthe Recommanded Movies for user(user_index={}:\n'.format(user_index))
    print(temp.head(num_recommendations))

In [53]:
user_index=4
num_recommendations=5
rec_items(user_index,pvt,pred,num_recommendations)


the Recommanded Movies for user(user_index=4:

                   user_ratings  user_predictions
Recommended Movie                                
Movie162                    0.0          0.019604
Movie86                     0.0          0.011711
Movie185                    0.0          0.000109
Movie163                    0.0          0.000104
Movie95                     0.0          0.000089


In [54]:
user_index=3627
num_recommendations=5
rec_items(user_index,pvt,pred,num_recommendations)


the Recommanded Movies for user(user_index=3627:

                   user_ratings  user_predictions
Recommended Movie                                
Movie140                    0.0          0.018641
Movie127                    0.0          0.002835
Movie188                    0.0          0.000071
Movie189                    0.0          0.000071
Movie190                    0.0          0.000071


In [55]:
# Model Evaluation

In [56]:
pvt.mean().head()

Movies
Movie1      0.001031
Movie10     0.001031
Movie100    0.000825
Movie101    0.005157
Movie102    0.001650
dtype: float64

In [57]:
pred.mean().head()

Movies
Movie1     -1.143966e-16
Movie10    -2.371565e-19
Movie100    2.546512e-16
Movie101    2.818410e-16
Movie102    7.905065e-04
dtype: float64

In [60]:
rmse=pd.concat([pvt.mean(),pred.mean()],axis=1)
rmse.columns=['avg_ratings','avg_prediction']
rmse.shape

(206, 2)

In [61]:
rmse.head()

,avg_ratings,avg_prediction
Movies,,
Movie1,0.001031,-1.143966e-16
Movie10,0.001031,-2.371565e-19
Movie100,0.000825,2.546512e-16
Movie101,0.005157,2.818410e-16
Movie102,0.001650,7.905065e-04


In [62]:
RMSE=round((((rmse.avg_ratings-rmse.avg_prediction)**2).mean()**0.5),5)
print('\nRMSE SVD Model={}'.format(RMSE))


RMSE SVD Model=0.00669


In [63]:
#Testing


In [64]:
user_index=2527
num_recommendations=5
rec_items(user_index,pvt,pred,num_recommendations)


the Recommanded Movies for user(user_index=2527:

                   user_ratings  user_predictions
Recommended Movie                                
Movie107                    0.0          0.041839
Movie15                     0.0          0.017169
Movie86                     0.0          0.015365
Movie89                     0.0          0.000248
Movie105                    0.0          0.000189
